In [52]:
import pathlib
import json
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
import numpy as np

import scipy.stats


In [53]:
Record = namedtuple('Record', 
                    ['header', 'train_loss', 'test_loss'])

def parse_file(fname, to_skip=3):
    train_loss = []
    test_loss = []
    
    with open(fname, 'r') as f:
        for _ in range(1): f.readline()
        
        header = eval(f.readline())
        
        header = [x.split('=') for x in header if '=' in x]
        header = [[x[0][2:], x[1]] for x in header]
        header = dict(header)
        
        f.readline()

        for line in f:
            if line.startswith('#') or line.startswith('submitit') or 'tensor' in line: continue
            
            try:
                line = json.loads(line)
            except:
                print(line)
                return
            
            if 'mode' in line and line['mode'] == 'train':
                train_loss.append(line['acc'])
                continue
            if 'mode' in line and line['mode'] == 'test':
                test_loss.append(line['acc'])
                continue        

    return Record(header=header, train_loss=train_loss, test_loss=test_loss)                
            

In [54]:
def time_to_stop(log):
    import math
    for i, score in enumerate(log.train_loss):
        if score == 1.0:
            break
    assert not math.isnan(i)
    return i

def generalization(log):
    return log.test_loss[-1]

In [55]:
def get_language_name(datum):
    name = datum.header['language']
    #position_scrambled = False #datum.header['scramble_positions'] == '1'
    #loss = datum.header['loss_type']
    #mixer = str(datum.header['mixers'])
    return name #+ '-' + mixer #('-pos_scrambled' if position_scrambled else '')

In [62]:
paths = [
    #nv =31
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_05_44_19/',
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_06_29_17/'
    # nv 11
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_06_57_04/'
    # nv 17
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_07_03_46'
    # nv 17 shuffled data
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_07_12_13/'
    
    # nv 31 shuffled
    # lstm
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_07_26_14/',
    # gru
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_08_31_11/',
    # gru add
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_08_08_50_29/'
    
    # redo everything with test (lstm, gru, gru add)
    #'/private/home/kharitonov/nest/compositional_efficiency/2020_02_09_03_33_26/'
    
    # nv=100
    #'/checkpoint/rchaabouni/EGG/experiment1_gru_nv100/2020_02_14_09_43_43/'
    #'/checkpoint/rchaabouni/EGG/experiment1_gru_nv100/2020_02_16_02_50_13/'
    # nv=50
    #'/checkpoint/rchaabouni/EGG/experiment1_gru_nv50/2020_02_14_09_39_47/'
    # nv=10
    '/checkpoint/rchaabouni/EGG/experiment1_gru_nv10/2020_02_14_09_44_51'

]

data = []

for path in paths:
    for fname in pathlib.Path(path).rglob('*.out'):
        parsed = parse_file(fname, to_skip=1)
        data.append(parsed)

In [63]:
language2caption = {
    'identity': 'Identity',
    'rotated': 'Rotated',
}


for task in ["autoenc", "linear", "mixed"]:
    for receiver_cell in ['lstm']:
                filtered = [d for d in data if \
                            d.header['loss_type'] == task and \
                            d.header['receiver_cell'] == receiver_cell and \
                            d.train_loss[-1] == 1.0
                           ]
                if len(filtered)>0:
                    languages = sorted(list(set(get_language_name(x) for x in filtered)))

                    print("Task:", task)

                    for language in languages:
                        measurements = [d for d in filtered if get_language_name(d) == language]
                        n = len(measurements)
                        time = [time_to_stop(x) for x in measurements]

                        mean, std = np.mean(time), np.std(time) / np.sqrt(n)
                        print(f"\t{language} time, mean: {mean}, std: {std}")
                        
                        test_acc = [generalization(x) for x in measurements]
                        mean, std = np.mean(test_acc), np.std(test_acc) / np.sqrt(n)
                        print(f"\t{language} generalization, mean: {mean}, std: {std}")

                else:
                    print(f'no successful runs for {task}, {receiver_cell}')

no successful runs for autoenc, lstm
no successful runs for linear, lstm
no successful runs for mixed, lstm


In [64]:
language2caption = {
    'identity': 'Identity',
    'rotated': 'Rotated',
}


for task in ["autoenc", "linear", "mixed"]:
    for receiver_cell in ['gru']:
                filtered = [d for d in data if \
                            d.header['loss_type'] == task and \
                            d.header['receiver_cell'] == receiver_cell and \
                            d.train_loss[-1] == 1.0
                           ]
                if len(filtered)>0:
                    languages = sorted(list(set(get_language_name(x) for x in filtered)))

                    print("Task:", task)

                    for language in languages:
                        measurements = [d for d in filtered if get_language_name(d) == language]
                        n = len(measurements)
                        time = [time_to_stop(x) for x in measurements]

                        mean, std = np.mean(time), np.std(time) / np.sqrt(n)
                        print(f"\t{language} time, mean: {mean}, std: {std}")
                        
                        test_acc = [generalization(x) for x in measurements]
                        mean, std = np.mean(test_acc), np.std(test_acc) / np.sqrt(n)
                        print(f"\t{language} generalization, mean: {mean}, std: {std}")

                else:
                    print(f'no successful runs for {task}, {receiver_cell}')

Task: autoenc
	identity time, mean: 6.9, std: 0.26267851073127396
	identity generalization, mean: 0.4616008996963501, std: 0.039695075496094134
Task: linear
	identity time, mean: 12.3, std: 0.3754996671103717
	identity generalization, mean: 0.0095238097012043, std: 0.009035079197343974
	rotated time, mean: 12.8, std: 0.2756809750418044
	rotated generalization, mean: 0.8301860928535462, std: 0.026174717883672413
Task: mixed
	identity time, mean: 11.3, std: 0.3754996671103717
	identity generalization, mean: 0.0, std: 0.0
	rotated time, mean: 6.9, std: 0.26267851073127396
	rotated generalization, mean: 0.8421742916107178, std: 0.025277038260457633


In [43]:
def mean_std(data):
    means = []
    stds = []
    
    losses = [d.test_loss for d in data]
    
    for l in zip(*losses):
        means.append(np.mean(l))
        stds.append(np.std(l) / np.sqrt(len(l)))
        
    return means, stds

In [44]:
for slice, name, color in [
        (no_rotation, 'no rotation', 'r'), 
        (rotation, 'rotation', 'black'),
        (swap_data, 'swap', 'blue')]:
    means, stds = mean_std(slice)
    xs = list(range(1, len(means) + 1))
    #plt.plot(means[:3], label=name, color=color)
    plt.errorbar(x=xs[:20], y=means[:20], yerr=stds[:20], color=color, alpha=0.75, label=name)
plt.xlabel('Epoch')
plt.ylabel('MSE')

plt.legend()

NameError: name 'no_rotation' is not defined

In [171]:
mean_std(rotation)

NameError: name 'rotation' is not defined